### 쇼핑몰 크롤링
- 상품의 시세 파악을 위해서는 쇼핑몰 데이터를 추출하여 파악하는 것이 일반적임
- 특정 사이트를 선정하여 매일 상품의 가격 변환를 확인
- 사이트 선정 시 구성이 변하지 않고, 접속량이 일정 수준 이상인 사이트를  선정
- http://jolse.com/category/toners-mists/1019/page=1

#### 크롤링 작업 순서
(1) 첫 번째 페이지에서 상품 정보 추출   
    - 추천 상품 2개  
    - 일반 상품 20개  
    - 각 상품박스  
        - 상품명  
        - 상품가격   
            - 정상가  
            - 세일가  

(2) 여러 페이지 크롤링  
    - (1) 접속 & 파싱 함수  
    - (2) 1개의 상품 추출 함수  
    - (3) 한 페이지에서 상품 추출  
    - (4) 여러 페이지에서 추출  
        - base_url + page 번호(i)  

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [2]:
from urllib.request import urlopen
import bs4
import pandas as pd
import requests

In [3]:
url = 'http://jolse.com/category/toners-mists/1019/page=1'
html = urlopen(url)
# 파서 객체 생성
bs_obj = bs4.BeautifulSoup(html, 'html.parser') 

In [5]:
# bs_obj

In [7]:
# 페이지 구조
# 추천 상품 2개는 모든 페이지에 동일
# 일반 상품
# 추천 상품과 일반 상품 그룹을 구성하는 태그명 동일
# <ul class='prdList grid5'>

uls = bs_obj.findAll('ul', {'class':'prdList grid5'})
len(uls)

2

In [9]:
# uls[0]

In [10]:
#### (1) 첫 페이지에서 상품 정보 추출
# 상품 정보만 담고 있는 div 태그 사용 (상품 이미지 제외)
# div {'class' : 'description'}
boxes = bs_obj.findAll('div', {'class' : 'description'})
len(boxes) # 22 : 추천 상품 2개, 일반 상품 20개
boxes

22

[<div class="description">
 <span class="chk"><input class="ProductCompareClass xECPCNO_37796" type="checkbox"/></span>
 <span class="prd-brand"></span>
 <strong class="name"><a class="" href="/product/haruharu-wonder-black-rice-hyaluronic-toner-300ml/37796/category/1019/display/2/"><span class="title displaynone"><span style="font-size:14px;color:#555555;font-weight:bold;">Product</span> :</span> <span style="font-size:14px;color:#555555;font-weight:bold;">Haruharu WONDER Black Rice Hyaluronic Toner 300ml</span></a></strong>
 <ul class="xans-element- xans-product xans-product-listitem spec"><li class="xans-record-">
 <strong class="title displaynone"><span style="font-size:16px;color:#888888;">Price</span></strong> <span style="font-size:16px;color:#888888;text-decoration:line-through;">USD 37.00</span><span id="span_product_tax_type_text" style="text-decoration:line-through;"> </span></li>
 <li class="xans-record-">
 <strong class="title"><span style="font-size:20px;color:#ff2d46;fon

In [ ]:
# 추출할 요소 파악
# 상품명 : <strong class="name"> :<span>:</span><span>상품명</span>
# 정상 가격 / 세일 가격 : <span> 태그로 선택자 없음
# - description boxes 안에서 순서로 찾아감
# - 정상 가격 : ul에서 두 번째[1] <span>
# - 세일 가격 : ul에서 마지막[-1] <span>

#### (1) 상품명 추출

In [12]:
# 전체 상품명 출력
for box in boxes:
    strong_tag = box.find('strong', {'class':'name'})
    print(strong_tag.text.split(':')[1])
    # 구분자 :으로 분리해서 오른쪽 값 (인덱스로 두 번째 값 :[1])

 Haruharu WONDER Black Rice Hyaluronic Toner 300ml
 SOME BY MI Propolis B5 Glow Barrier Calming Toner 150ml
 Anua Heartleaf 77% Soothing Toner 250ml
 COSRX AHA/BHA CLARIFYING TREATMENT TONER 150ml
 COSRX Full Fit Propolis Synergy Toner 150ml
 Beauty of Joseon Ginseng Essence Water 150ml
 Haruharu WONDER Black Rice Hyaluronic Toner 150ml (Fragrance Free)
 ROUND LAB 1025 Dokdo Toner 200ml
 SOME BY MI AHA BHA PHA 30 Days Miracle Toner 150ml
 COSRX BHA BLACKHEAD POWER LIQUID 100 ML
 Pyunkang Yul Essence Toner 200ml
 🔸0.99deal🔸 COSRX Refresh AHA/BHA Vitamin C Daily Toner 50ml
 numbuzin No.3 Super Glowing Essence Toner 200ml
 Isntree Green tea Fresh Toner 200ml
 AXIS-Y The Mini Glow Set
 COSRX CENTELLA WATER ALCOHOL FREE TONER 150ml
 COSRX AHA 7 WHITEHEAD POWER LIQUID 100ml
 COSRX Balancium Comfort Ceramide Cream Mist 120ml
 ROUND LAB 1025 Dokdo Toner 100ml
 Isntree Hyaluronic Acid Toner 200ml (Renewal)
 ROUND LAB 1025 Dokdo Toner 500ml
 COSRX Full Fit Propolis Synergy Toner 50ml


In [13]:
# 상품명을 리스트로 저장
product_list = []

for box in boxes:
    strong_tag = box.find('strong', {'class':'name'})
    product_list.append(strong_tag.text.split(':')[1])

In [14]:
len(product_list)

22

In [16]:
# product_list

#### (2) 상품 가격 추출

In [17]:
# 상품 가격의 <span> 태그가 선택자로 지정되어 있지 않기 때문에
# description boxes 안에서 찾아감
# 먼저 ul 태그 확인
boxes[0].find('ul')

<ul class="xans-element- xans-product xans-product-listitem spec"><li class="xans-record-">
<strong class="title displaynone"><span style="font-size:16px;color:#888888;">Price</span></strong> <span style="font-size:16px;color:#888888;text-decoration:line-through;">USD 37.00</span><span id="span_product_tax_type_text" style="text-decoration:line-through;"> </span></li>
<li class="xans-record-">
<strong class="title"><span style="font-size:20px;color:#ff2d46;font-weight:bold;"></span></strong> <span style="font-size:20px;color:#ff2d46;font-weight:bold;">USD 33.30</span></li>
</ul>

In [ ]:
# <ul class="xans-element- xans-product xans-product-listitem spec">
# <li class="xans-record-">
# <strong class="title displaynone">
# <span style="font-size:16px;color:#888888;">Price</span></strong> 
# 정상가(두 번째) : <span style="font-size:16px;color:#888888;text-decoration:line-through;">USD 37.00</span>
# <span id="span_product_tax_type_text" style="text-decoration:line-through;"> </span></li>
# <li class="xans-record-">
# <strong class="title"><span style="font-size:20px;color:#ff2d46;font-weight:bold;"></span>
# 세일가(마지막) : </strong> <span style="font-size:20px;color:#ff2d46;font-weight:bold;">USD 33.30</span></li>
# </ul>

In [18]:
# 정상 가격 : 두 번째 span
boxes[0].find('ul').findAll('span')[1].text

# 세일 가격 : 마지막[-1] span
boxes[0].find('ul').findAll('span')[-1].text

'USD 37.00'

'USD 33.30'

In [20]:
# 첫 페이지의 전체 상품 정보 추출
# 상품명, 정상가격, 세일가격
for box in boxes:
    name = box.find('strong', {'class':'name'}).text.split(':')[1]
    price = box.find('ul').findAll('span')[1].text
    sale_price = box.find('ul').findAll('span')[-1].text

    print('상품명 : ', name[1:])
    print('정상가 : ', price)
    print('할인가 : ', sale_price)

상품명 :  Haruharu WONDER Black Rice Hyaluronic Toner 300ml
정상가 :  USD 37.00
할인가 :  USD 33.30
상품명 :  SOME BY MI Propolis B5 Glow Barrier Calming Toner 150ml
정상가 :  USD 22.00
할인가 :  USD 19.80
상품명 :  Anua Heartleaf 77% Soothing Toner 250ml
정상가 :  USD 28.00
할인가 :  USD 25.20
상품명 :  COSRX AHA/BHA CLARIFYING TREATMENT TONER 150ml
정상가 :  USD 17.25
할인가 :  USD 8.29
상품명 :  COSRX Full Fit Propolis Synergy Toner 150ml
정상가 :  USD 28.00
할인가 :  USD 10.59
상품명 :  Beauty of Joseon Ginseng Essence Water 150ml
정상가 :  USD 24.55
할인가 :  USD 18.41
상품명 :  Haruharu WONDER Black Rice Hyaluronic Toner 150ml (Fragrance Free)
정상가 :  USD 22.00
할인가 :  USD 19.80
상품명 :  ROUND LAB 1025 Dokdo Toner 200ml
정상가 :  USD 17.00
할인가 :  USD 11.99
상품명 :  SOME BY MI AHA BHA PHA 30 Days Miracle Toner 150ml
정상가 :  USD 24.00
할인가 :  USD 14.29
상품명 :  COSRX BHA BLACKHEAD POWER LIQUID 100 ML
정상가 :  USD 20.81
할인가 :  USD 11.39
상품명 :  Pyunkang Yul Essence Toner 200ml
정상가 :  USD 21.90
할인가 :  USD 9.39
상품명 :  🔸0.99deal🔸 COSRX Refresh AHA/BHA Vitam

In [ ]:
#################################################

#### 첫 페이지에서 상품 정보 추출하여 df로 저장
- 접속 / 파싱  
- 수집  
- df로 저장  

In [21]:
url = 'http://jolse.com/category/toners-mists/1019/page=1'
html = urlopen(url)
# 파서 객체 생성
bs_obj = bs4.BeautifulSoup(html, 'html.parser') 

In [22]:
# 빈 리스트  생성
prd_list = []
price_list = []
sale_price_list = []

In [23]:
# 전체 상품 데이터 추출 : boxes 추출
boxes = bs_obj.findAll('div', {'class' : 'description'})

# 각 box에서 상품명, 정상가, 세일가 추출해서 리스트에 저장
for box in boxes:
    prd_list.append(box.find('strong', {'class':'name'}).text.split(':')[1])
    price_list.append(box.find('ul').findAll('span')[1].text)
    sale_price_list.append(box.find('ul').findAll('span')[-1].text)

In [24]:
# 각 리스트를 데이터프레임으로 생성
product_df = pd.DataFrame({
    '품목' : prd_list,
    '가격' : price_list,
    '세일가격' : sale_price_list
})

product_df

,품목,가격,세일가격
0,Haruharu WONDER Black Rice Hyaluronic Toner 3...,USD 37.00,USD 33.30
1,SOME BY MI Propolis B5 Glow Barrier Calming T...,USD 22.00,USD 19.80
2,Anua Heartleaf 77% Soothing Toner 250ml,USD 28.00,USD 25.20
3,COSRX AHA/BHA CLARIFYING TREATMENT TONER 150ml,USD 17.25,USD 8.29
4,COSRX Full Fit Propolis Synergy Toner 150ml,USD 28.00,USD 10.59
5,Beauty of Joseon Ginseng Essence Water 150ml,USD 24.55,USD 18.41
6,Haruharu WONDER Black Rice Hyaluronic Toner 1...,USD 22.00,USD 19.80
7,ROUND LAB 1025 Dokdo Toner 200ml,USD 17.00,USD 11.99
8,SOME BY MI AHA BHA PHA 30 Days Miracle Toner ...,USD 24.00,USD 14.29
9,COSRX BHA BLACKHEAD POWER LIQUID 100 ML,USD 20.81,USD 11.39


In [ ]:
######################### 첫 페이지(한 페이지)에서 상품 정보 추출 완료

#### (2) 여러 페이지 크롤링
- 현재 페이지 뿐 아니라 다른 모든 페이지에서도 반복 적용할 수 있도록 함수로 작성

####  함수명 및 수행 기능   
(1) get_request_product(url) : 접속 및 파싱  
    - url 받아서 접속 및 파싱  
    - 반환값 : bs4 객체  
(2) get_product_info(box) : 1개 상품 정보 추출  
    - box 받아서 상품 정보 추출 후 반환  
    - 반환값 : 1개 상품의 상품명/가격/세일가격을 딕셔너리 형태로 반환  
(3) get_page_product(url) : 각 페이지에서 전체 상품 정보 추출하고 df에 추가  
    - 전달받은 url 페이지에서 box 추출하여  
    - get_product_info(box) 호출해서 전달하고 반환된 1개의 상품 정보로  
    - 데이터프레임 생성 및 추가  

In [40]:
# 최종 상품 정보 저장할 빈 데이터프레임 생성
all_product_df = pd.DataFrame({
    '품목' : [],
    '가격' : [],
    '세일가격' : []
})

all_product_df

,품목,가격,세일가격


In [31]:
# (1) 접속 및 파싱 기능 수행하는 함수
# url 받아서 접속 및 파싱
# bs4 객체 반환

def get_request_product(url):
    try:
        html = urlopen(url)
        # 파서 객체 생성
        bs_obj = bs4.BeautifulSoup(html, 'html.parser') 
    except:
        print("접속 및 파싱 오류")

    return bs_obj

In [32]:
# (2) 1개의 상품 정보를 추출하는 함수
# 1개 상품 정보 추출
# box 받아서 상품 정보 추출 후 반환
# 반환값: 1개 상품의 상품명/가격/세일가격을 딕셔너리 형태로 반환

def get_product_info(box) : 
    try:
        name = box.find('strong', {'class':'name'}).text.split(':')[1]
        price = box.find('ul').findAll('span')[1].text
        sale_price = box.find('ul').findAll('span')[-1].text
    except:
        print("상품 정보 추출 오류")

    return {'품목':name, '가격':price, '세일가격':sale_price}

In [33]:
# (3) 전달 받은 url 한 페이지에서 상품 정보 추출하고 df에 추가하는 함수
def get_page_product(url):
    global all_product_df
    print(url)
    try:
        # 접속 및 파싱
        bs_obj = get_request_product(url)

        # 페이지에서 전체 상품 추출
        boxes = bs_obj.findAll('div', {'class' : 'description'})
        # 첫 페이지에만 추천 상품 2개 포함되어 있으므로
        # 첫 페이지에서는 22개 다 추출하고
        # 두 번째 이후 페이지에서 추천 상품 제외하고 추출
        # 두 번째 페이지 구분 : https:// ...... /?page=2
        # url을 = 구분자로 split() 해서 오른쪽 값([1])이 페이지  번호
        # 페이지 번호가 1이 아니라면 세 번째 상품부터 추출
        if url.split('=')[1] != '1':  # 페이지 번호가 1이 아니라면 
            boxes = boxes[2:]         # 세 번째 상품부터 끝까지 추출
        
    except:
        print("페이지 정보 추출 오류")
        
    # 추출된 각  상품 정보를 df에 추가
    for box in boxes:
        df = pd.DataFrame(get_product_info(box), index=range(1,2)) # 형식적 인덱스 추가
        all_product_df = pd.concat([all_product_df, df], axis=0, ignore_index=True)
        

In [38]:
# 테스트 : 테스트 후  주석 처리할 것
# url = 'http://jolse.com/category/toners-mists/1019/page=2'
# get_page_product(url) 

http://jolse.com/category/toners-mists/1019/page=2


In [42]:
# all_product_df
# 결과 확인 후
# 데이터프레임을 다시 초기화 할 것

#### 여러 페이지 추출  
- url 뒤에 페이지 번호 있음  
- url 작성 시 페이지 번호를 따로 붙임   
- 'http://jolse.com/category/toners-mists/1019/page=' + 페이지 번호  

In [41]:
# 마지막 페이지 값 추출
url = 'http://jolse.com/category/toners-mists/1019/page=1'
html = urlopen(url)
# 파서 객체 생성
bs_obj = bs4.BeautifulSoup(html, 'html.parser') 
last_page = bs_obj.find('a', {'class':'last'})['href'].split('=')[1]
last_page

'22'

In [43]:
# 모든 페이지에서 상품 정보 추출
base_url = 'http://jolse.com/category/toners-mists/1019/page='

for i in range(1, int(last_page) + 1):
    url = base_url + str(i)
    get_page_product(url) 

http://jolse.com/category/toners-mists/1019/page=1
http://jolse.com/category/toners-mists/1019/page=2
http://jolse.com/category/toners-mists/1019/page=3
http://jolse.com/category/toners-mists/1019/page=4
http://jolse.com/category/toners-mists/1019/page=5
http://jolse.com/category/toners-mists/1019/page=6
http://jolse.com/category/toners-mists/1019/page=7
http://jolse.com/category/toners-mists/1019/page=8
http://jolse.com/category/toners-mists/1019/page=9
http://jolse.com/category/toners-mists/1019/page=10
http://jolse.com/category/toners-mists/1019/page=11
http://jolse.com/category/toners-mists/1019/page=12
http://jolse.com/category/toners-mists/1019/page=13
http://jolse.com/category/toners-mists/1019/page=14
http://jolse.com/category/toners-mists/1019/page=15
http://jolse.com/category/toners-mists/1019/page=16
http://jolse.com/category/toners-mists/1019/page=17
http://jolse.com/category/toners-mists/1019/page=18
http://jolse.com/category/toners-mists/1019/page=19
http://jolse.com/cate

In [44]:
all_product_df

,품목,가격,세일가격
0,Haruharu WONDER Black Rice Hyaluronic Toner 3...,USD 37.00,USD 33.30
1,SOME BY MI Propolis B5 Glow Barrier Calming T...,USD 22.00,USD 19.80
2,Anua Heartleaf 77% Soothing Toner 250ml,USD 28.00,USD 25.20
3,COSRX AHA/BHA CLARIFYING TREATMENT TONER 150ml,USD 17.25,USD 8.29
4,COSRX Full Fit Propolis Synergy Toner 150ml,USD 28.00,USD 10.59
...,...,...,...
437,COSRX Balancium Comfort Ceramide Cream Mist 1...,USD 29.00,USD 26.10
438,ROUND LAB 1025 Dokdo Toner 100ml,USD 13.55,USD 12.19
439,Isntree Hyaluronic Acid Toner 200ml (Renewal),USD 19.30,USD 17.37
440,ROUND LAB 1025 Dokdo Toner 500ml,USD 34.00,USD 30.60


In [45]:
all_product_df.to_csv('./crawl_data/shopping_products.csv', index=0)

In [46]:
all_product_df = pd.read_csv('./crawl_data/shopping_products.csv')
# all_product_df

,품목,가격,세일가격
0,Haruharu WONDER Black Rice Hyaluronic Toner 3...,USD 37.00,USD 33.30
1,SOME BY MI Propolis B5 Glow Barrier Calming T...,USD 22.00,USD 19.80
2,Anua Heartleaf 77% Soothing Toner 250ml,USD 28.00,USD 25.20
3,COSRX AHA/BHA CLARIFYING TREATMENT TONER 150ml,USD 17.25,USD 8.29
4,COSRX Full Fit Propolis Synergy Toner 150ml,USD 28.00,USD 10.59
...,...,...,...
437,COSRX Balancium Comfort Ceramide Cream Mist 1...,USD 29.00,USD 26.10
438,ROUND LAB 1025 Dokdo Toner 100ml,USD 13.55,USD 12.19
439,Isntree Hyaluronic Acid Toner 200ml (Renewal),USD 19.30,USD 17.37
440,ROUND LAB 1025 Dokdo Toner 500ml,USD 34.00,USD 30.60
